In [11]:
import requests
import pandas as pd
import io
import datetime as dt
import plotly.express as px
import ipywidgets as widgets
import altair as alt

url = "https://onemocneni-aktualne.mzcr.cz/api/v2/covid-19/osoby.csv"
s = requests.get(url).content
# parse_dates = [0], index_col = 0
df = pd.read_csv(io.StringIO(s.decode('utf-8')), parse_dates=[0])
okresy = {'CZZZZZ': 'Extra-Regio',
               'CZ0100': 'Praha',
               'CZ020A': 'Praha-západ',
               'CZ020B': 'Příbram',
               'CZ020C': 'Rakovník',
               'CZ0201': 'Benešov',
               'CZ0202': 'Beroun',
               'CZ0203': 'Kladno',
               'CZ0204': 'Kolín',
               'CZ0205': 'Kutná Hora',
               'CZ0206': 'Mělník',
               'CZ0207': 'Mladá Boleslav',
               'CZ0208': 'Nymburk',
               'CZ0209': 'Praha-východ',
               'CZ0311': 'České Budějovice',
               'CZ0312': 'Český Krumlov',
               'CZ0313': 'Jindřichův Hradec',
               'CZ0314': 'Písek',
               'CZ0315': 'Prachatice',
               'CZ0316': 'Strakonice',
               'CZ0317': 'Tábor',
               'CZ0321': 'Domažlice',
               'CZ0322': 'Klatovy',
               'CZ0323': 'Plzeň-město',
               'CZ0324': 'Plzeň-jih',
               'CZ0325': 'Plzeň-sever',
               'CZ0326': 'Rokycany',
               'CZ0327': 'Tachov',
               'CZ0411': 'Cheb',
               'CZ0412': 'Karlovy Vary',
               'CZ0413': 'Sokolov',
               'CZ0421': 'Děčín',
               'CZ0422': 'Chomutov',
               'CZ0423': 'Litoměřice',
               'CZ0424': 'Louny',
               'CZ0425': 'Most',
               'CZ0426': 'Teplice',
               'CZ0427': 'Ústí nad Labem',
               'CZ0511': 'Česká Lípa',
               'CZ0512': 'Jablonec nad Nisou',
               'CZ0513': 'Liberec',
               'CZ0514': 'Semily',
               'CZ0521': 'Hradec Králové',
               'CZ0522': 'Jičín',
               'CZ0523': 'Náchod',
               'CZ0524': 'Rychnov nad Kněžnou',
               'CZ0525': 'Trutnov',
               'CZ0531': 'Chrudim',
               'CZ0532': 'Pardubice',
               'CZ0533': 'Svitavy',
               'CZ0534': 'Ústí nad Orlicí',
               'CZ0631': 'Havlíčkův Brod',
               'CZ0632': 'Jihlava',
               'CZ0633': 'Pelhřimov',
               'CZ0634': 'Třebíč',
               'CZ0635': 'Žďár nad Sázavou',
               'CZ0641': 'Blansko',
               'CZ0642': 'Brno-město',
               'CZ0643': 'Brno-venkov',
               'CZ0644': 'Břeclav',
               'CZ0645': 'Hodonín',
               'CZ0646': 'Vyškov',
               'CZ0647': 'Znojmo',
               'CZ0711': 'Jeseník',
               'CZ0712': 'Olomouc',
               'CZ0713': 'Prostějov',
               'CZ0714': 'Přerov',
               'CZ0715': 'Šumperk',
               'CZ0721': 'Kroměříž',
               'CZ0722': 'Uherské Hradiště',
               'CZ0723': 'Vsetín',
               'CZ0724': 'Zlín',
               'CZ0801': 'Bruntál',
               'CZ0802': 'Frýdek-Místek',
               'CZ0803': 'Karviná',
               'CZ0804': 'Nový Jičín',
               'CZ0805': 'Opava',
               'CZ0806': 'Ostrava-město'}

def count_by_date_district(df):
    date_district_count = df.groupby(['datum', 'okres_lau_kod', 'okres_nazev']).count().reset_index()
    return date_district_count

# Týdenní přírůstek v okresech po dnech'

def week_count_by_date_district(df):
    date_district_count = df.copy()
    date_district_count = date_district_count.groupby(
        ['datum', 'okres_lau_kod', 'okres_nazev']).count().reset_index()
    range_max = date_district_count['datum'].max()
    range_min = range_max - dt.timedelta(days=7)
    date_district_count = date_district_count[(date_district_count['datum'] >= range_min) &
                                              (date_district_count['datum'] <= range_max)]
    date_district_count['animate_date'] = date_district_count['datum']
    date_district_count['animate_date'] = date_district_count['animate_date'].dt.strftime('%Y-%m-%d')
    return date_district_count

# Celkový týdenní přírůstek v okresech

def week_total_count_by_district(df):
    week_total_count_district = df.copy()
    range_max = week_total_count_district['datum'].max()
    range_min = range_max - dt.timedelta(days=7)
    week_total_count_district = week_total_count_district[
        (week_total_count_district['datum'] >= range_min) & (week_total_count_district['datum'] <= range_max)]
    week_total_count_district = week_total_count_district.groupby(
        ['okres_lau_kod', 'okres_nazev']).count().reset_index()
    return week_total_count_district


def total_count_by_district(df):
    total_count_district = df.copy()
    total_count_district = total_count_district.groupby(['okres_lau_kod', 'okres_nazev']).count().reset_index()
    return total_count_district

def total_count_by_date(df):
    total = df.groupby(['datum']).count().reset_index()
    return total


df['okres_nazev'] = df['okres_lau_kod'].map(okresy)
date_district_count = count_by_date_district(df)
week_date_district_count = week_count_by_date_district(df)
week_total_count_district = week_total_count_by_district(df)
total_count_district = total_count_by_district(df)
total_count_date = total_count_by_date(df)

In [6]:
republic_total_line_graph = px.line(total_count_date, x='datum', y='vek',
                                    labels={'datum': 'Datum', 'vek': 'Počet'}, title='Přírůstky celá ČR',
                                    template='plotly', color_discrete_sequence=px.colors.diverging.Spectral)

republic_total_line_graph.show()

In [32]:
alt.Chart(total_count_date).mark_line().encode(
    x='datum',
    y='vek',
)

alt.Chart(...)

In [17]:
total_count_date

,datum,vek,pohlavi,kraj_nuts_kod,okres_lau_kod,nakaza_v_zahranici,nakaza_zeme_csu_kod,okres_nazev
0,2020-03-01,3,3,3,3,3,3,3
1,2020-03-03,1,1,1,1,1,1,1
2,2020-03-04,1,1,1,1,1,1,1
3,2020-03-05,3,3,3,3,2,2,3
4,2020-03-06,11,11,11,11,8,8,11
...,...,...,...,...,...,...,...,...
231,2020-10-19,7815,7815,7815,7815,43,43,7815
232,2020-10-20,11626,11626,11626,11626,35,35,11626
233,2020-10-21,14446,14446,14446,14446,41,41,14446
234,2020-10-22,13671,13671,13671,13671,21,21,13671


In [36]:
date_district_count[date_district_count['okres_lau_kod'].isin(['CZ0100','CZ0722'])]

,datum,okres_lau_kod,okres_nazev,vek,pohlavi,kraj_nuts_kod,nakaza_v_zahranici,nakaza_zeme_csu_kod
0,2020-03-01,CZ0100,Praha,2,2,2,2,2
3,2020-03-04,CZ0100,Praha,1,1,1,1,1
5,2020-03-06,CZ0100,Praha,8,8,8,7,7
8,2020-03-07,CZ0100,Praha,4,4,4,3,3
17,2020-03-09,CZ0100,Praha,4,4,4,4,4
...,...,...,...,...,...,...,...,...
8928,2020-10-21,CZ0722,Uherské Hradiště,274,274,274,1,1
8937,2020-10-22,CZ0100,Praha,1670,1670,1670,7,7
9005,2020-10-22,CZ0722,Uherské Hradiště,243,243,243,0,0
9014,2020-10-23,CZ0100,Praha,579,579,579,0,0


In [4]:
okres_line = px.line(date_district_count[date_district_count['okres_lau_kod'].isin(['CZ0100','CZ0722'])],
                     x='datum',
                     y='vek', labels={'datum': 'Datum', 'vek': 'Počet', 'okres_nazev': 'Název okresu'},
                     title='Přírůstky v okresech',
                     template='plotly', color='okres_nazev',
                     color_discrete_sequence=px.colors.diverging.Spectral)
okres_line.show()

In [37]:
alt.Chart(date_district_count[date_district_count['okres_lau_kod'].isin(['CZ0100','CZ0722'])]).mark_line().encode(
    x='datum',
    y='vek',
)

alt.Chart(...)